## Import Required Libraries

In [1]:
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
from keras.optimizers import Adam
from joblib import dump
from keras.metrics import CategoricalAccuracy
import numpy as np

## Load data

In [2]:
train_dir = './animals-detection-images-dataset/train'
test_dir = "./animals-detection-images-dataset/test"
categories = ['Bear', 'Brown bear', 'Bull', 'Butterfly', 'Camel', 'Canary', 'Caterpillar', 'Cattle', 'Centipede',
              'Cheetah', 'Chicken', 'Crab', 'Crocodile', 'Deer', 'Duck', 'Eagle', 'Elephant', 'Fish', 'Fox',
              'Frog', 'Giraffe', 'Goat', 'Goldfish', 'Goose', 'Hamster', 'Harbor Seal', 'Hedgehog', 'Hippopotamus',
              'Horse', 'Jaguar', 'Jellyfish', 'Kangaroo', 'Koala', 'Ladybug', 'Leopard', 'Lion', 'Lizard',
              'Lynx', 'Magpie', 'Monkey', 'Moths and butterflies', 'Mouse', 'Mule', 'Ostrich', 'Otter', 'Owl',
              'Panda', 'Parrot', 'Penguin', 'Pig', 'Polar bear', 'Rabbit', 'Raccoon', 'Raven', 'Red panda',
              'Rhinoceros', 'Scorpion', 'Sea lion', 'Sea turtle', 'Seahorse', 'Shark', 'Sheep', 'Shrimp',
              'Snail', 'Snake', 'Sparrow', 'Spider', 'Squid', 'Squirrel', 'Starfish', 'Swan', 'Tick', 'Tiger',
              'Tortoise', 'Turkey', 'Turtle', 'Whale', 'Woodpecker', 'Worm', 'Zebra']

data = []
labels = []

for category in categories: 
    category_path = os.path.join(train_dir, category)
    for file in os.listdir(category_path):
        img_path = os.path.join(category_path, file)
        img = load_img(img_path, target_size=(224, 224))
        img = np.asarray(img)
        data.append(img)
        labels.append(category)


labels = np.asarray(labels)
label_as_binary = LabelBinarizer()
train__labels = label_as_binary.fit_transform(labels)


## Load the pre-trained model and add a classifier on top of the base model

In [29]:
model = Sequential([
    Conv2D(16, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2)])
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))



model.summary()


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x0000028456C62560>
Traceback (most recent call last):
  File "c:\Users\charl\anaconda3\lib\weakref.py", line 371, in remove
    self = selfref()
KeyboardInterrupt: 


## Define training parameters and callbacks

In [25]:
adam = Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[CategoricalAccuracy()], run_eagerly=True)
bs = 30
train_dir = './animals-detection-images-dataset/train'
test_dir = "./animals-detection-images-dataset/test"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
test_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=bs, class_mode='categorical', target_size=(224, 224))
validation_generator = test_datagen.flow_from_directory(test_dir, batch_size=bs, class_mode='categorical', target_size=(224, 224))




Found 22565 images belonging to 80 classes.
Found 6505 images belonging to 80 classes.


## Fiting the model

In [26]:
history = model.train_on_batch(train_datagen, train__labels)

Epoch 1/30


ValueError: Shapes (30, 80) and (30, 10) are incompatible

## Evaluate model

In [ ]:
label_pred = model.predict(test_data)
label_pred = np.argmax(label_pred, axis=1)
label_true = np.argmax(test_labels, axis=1)

accuracy = accuracy_score(label_true, label_pred)
precision = precision_score(label_true, label_pred, average='weighted')
recall = recall_score(label_true, label_pred, average='weighted')
f1 = f1_score(label_true, label_pred, average='weighted')

print('Accuracy: {:.2f}%, Precision: {:.2f}%, Recall: {:.2f}%, F1 Score: {:.2f}%'.format(accuracy * 100,
                                                                                         precision * 100,
                                                                                         recall * 100,
                                                                                         f1 * 100))

## Save model

In [ ]:
dump(model, 'Animals_prediction_model.joblib')